# [Module 2.0] Inferencde Pipeline 생성

- 이 노트북에서는 아래의 내용을 진행 합니다.
    - Feature Transfomer(전처리 학습 모델) 생성
    - Train 데이타를 Feature Transfomer를 통해서 전처리 데이타 생성
    - Validation 데이타를 Feature Transfomer를 통해서 전처리 데이타 생성
    - XGBoost를 학습
    - Post-Processing Model 생성
    - Inference Pipeline 생성
- 소요 시간은 약 10분 걸립니다.

## Feature Transformer (전처리 학습 모델) - preprocessing.py 파일
- Numerical 데이타는 <a href=https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html>StandardScaler</a>를 사용하여 Normalization을 함. 
    * z = (x - u) / s 
        * (z: 표준화된 값. 이 값을 학습시에 사용, x: 각 테이타의 값, u: 데이타 항목의 평균, s: 데이타 항목의 평균편차)
- 아래 Account Length, ..CustServ Calls까지 모두 위의 방법으로 전처리 함.

```python
    numeric_features = list([
    'Account Length',
    'VMail Message',
    'Day Mins',
    'Day Calls',
    'Eve Mins',
    'Eve Calls',
    'Night Mins',
    'Night Calls',
    'Intl Mins',
    'Intl Calls',
    'CustServ Calls'])

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])
```
- Categorical 데이타는 One Hot Encoding 방식으로 전처리 함.
    - State, Area Code, Int'l Plan, VMail Plan을 적용 함
```python
    categorical_features = ['State','Area Code',"Int'l Plan",'VMail Plan']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
```
- 최종적으로 Numerical and Categorical Transformer를 합쳐서 Transformer 생성하고, 학습하여 Transformer의 모델을 S3에 업로드 함.
```python
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)],
        remainder="drop")

    preprocessor.fit(concat_data)

    joblib.dump(preprocessor, os.path.join(args.model_dir, "model.joblib"))
```
- Phone 데이타 항목은 위의 전처리 항목에서 제외 함. 유저별로 고유한 번호이기에 피쳐로서 의미가 없을 것으로 보임

In [1]:
import sagemaker
import pandas as pd
import numpy as np
import os
import time
import json
from time import strftime, gmtime

In [2]:
%store -r

Unable to restore variable 'sklearn_preprocessor', ignoring (use %store -d to forget!)
The error was: <class 'KeyError'>


## Feature Transformer (전처리 학습 모델) 생성
아래는 다음과 같은 작업을 합니다.
- SKLearn 이라는 Estimator를 생성 합니다. 
    - s3_input_train의 학습 데이타를 SKLearn 입력으로 제공 합니다.
    - "전처리 학습 모델 (Featurizer)" 을 생성할 수 있는 소스 코드를 preprocessing.py 지정 합니다. 
    - 사용할 리소스로 instance_type = 'local' 를 지정 합니다. (이미 노트북 인스턴스에 설치된 Docker-compose를 이용 합니다.)
        - Local 이 아니라 SageMaker Cloud Instance도 사용 가능 합니다. (예: ml.m4.xlarge)
- SKLearn의 "전처리 학습 모델"이 완료가 되면 결과인 모델 아티펙트인 (model.tar.gz) 파일이  s3://{bucket_name}/{job_name}/output.tar.gz 에 저장 됩니다. 
    - (예: s3://sagemaker-us-east-2-057716757052/sagemaker-scikit-learn-2020-07-15-08-39-41-035/model.tar.gz)

In [3]:
from sagemaker.sklearn.estimator import SKLearn
sagemaker_session = sagemaker.Session()
from sagemaker import get_execution_role

role = get_execution_role()

script_path = 'preprocessing.py'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    train_instance_type="local")
sklearn_preprocessor.fit({'train': s3_input_train})

Creating tmpaatp0cdr_algo-1-2a0b0_1 ... 
Attaching to tmpaatp0cdr_algo-1-2a0b0_12mdone
algo-1-2a0b0_1  | 2020-07-19 01:59:33,378 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
algo-1-2a0b0_1  | 2020-07-19 01:59:33,380 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-2a0b0_1  | 2020-07-19 01:59:33,389 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-2a0b0_1  | 2020-07-19 01:59:33,514 sagemaker-containers INFO     Module preprocessing does not provide a setup.py. 
algo-1-2a0b0_1  | Generating setup.py
algo-1-2a0b0_1  | 2020-07-19 01:59:33,514 sagemaker-containers INFO     Generating setup.cfg
algo-1-2a0b0_1  | 2020-07-19 01:59:33,514 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-2a0b0_1  | 2020-07-19 01:59:33,514 sagemaker-containers INFO     Installing module with the following command:
algo-1-2a0b0_1  | /miniconda3/bin/python -m pip install . 
algo-1-2a0b0_1  | Pro

## Feature Transfomer를 사용하여 전처리된 학습 및 검증 데이타 생성 

![Transformer_Train](img/Fig2.1.transformer_train.png)

#### Preprocessed Training data (Feature) 만들기

In [4]:
# 아웃풋 경로 지정
transform_train_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-train-output')
instance_type = 'local'

# scikit_learn_inferencee_model 이름으로 전처리 학습 모델 생성
# TRANSFORM_MODE의 환경 변수는 전처리 모드라는 것을 알려 줌.
    # 추론시에는 환경 변수를 TRANSFORM_MODE": "inverse-label-transform" 설정 함.
    # 위의 두개의 과정을 분리할 수 있으나, 한개의 소스를 (preprocessor.py)를 사용하기 위해서, 환경 변수를 통해서 구분함.
scikit_learn_inferencee_model = sklearn_preprocessor.create_model(
    env={'TRANSFORM_MODE': 'feature-transform'})
# scikit_learn_inferencee_model 에서 Train Transformer 생성
transformer_train = scikit_learn_inferencee_model.transformer(
    instance_count=1, 
    instance_type= instance_type,
    assemble_with = 'Line',
    output_path = transform_train_output_path,
    accept = 'text/csv')


# Preprocess training input
transformer_train.transform(s3_input_train.config['DataSource']['S3DataSource']['S3Uri'], 
                            content_type='text/csv')
print('Waiting for transform job: ' + transformer_train.latest_transform_job.job_name)
transformer_train.wait()
preprocessed_train_path = transformer_train.output_path + transformer_train.latest_transform_job.job_name
print(preprocessed_train_path)

Attaching to tmpfhwps_sm_algo-1-66tkn_1
algo-1-66tkn_1  | Processing /opt/ml/code
algo-1-66tkn_1  | Building wheels for collected packages: preprocessing
algo-1-66tkn_1  |   Building wheel for preprocessing (setup.py) ... done
algo-1-66tkn_1  |   Created wheel for preprocessing: filename=preprocessing-1.0.0-py2.py3-none-any.whl size=9700 sha256=dc5002d37b088573e536d00abaaa1aa58d5db880c1bd116202dccaf715a50d0a
algo-1-66tkn_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-1wdr2csn/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-66tkn_1  | Successfully built preprocessing
algo-1-66tkn_1  | Installing collected packages: preprocessing
algo-1-66tkn_1  | Successfully installed preprocessing-1.0.0
algo-1-66tkn_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-66tkn_1  |   

#### Training 전처리된 학습 파일 확인

In [5]:
print(preprocessed_train_path)

s3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-07-19-01-59-2020-07-19-01-59-35-904


In [6]:
! aws s3 ls {preprocessed_train_path} --recursive

2020-07-19 01:59:42    1054526 sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-07-19-01-59-2020-07-19-01-59-35-904/train.csv.out


In [7]:
preprocessed_train_path_file = os.path.join (preprocessed_train_path, 'train.csv.out')
df_pre_train = pd.read_csv(preprocessed_train_path_file)
df_pre_train.head()


,0.0,0.11941369588439606,-0.5962380254245051,1.744368057672484,0.9789570533336895,-0.028992907038264654,-0.8931854019845896,-0.8017032037830547,-1.9825286353116254,-1.5305589315744583,...,0.0.48,0.0.49,0.0.50,0.0.51,0.0.52,0.0.53,1.0.1,0.0.54,1.0.2,0.0.55
0,0.0,-1.852652,-0.596238,0.140284,-0.310405,0.970689,-0.689888,0.146389,1.232901,0.124852,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,1.181295,-0.596238,1.835130,0.185503,0.030988,-0.639063,1.568529,-0.063643,-0.846802,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.776769,-0.596238,0.216227,0.334276,0.136954,1.393914,1.394712,-0.634123,0.844596,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,-0.234547,1.508734,-0.459859,0.483049,-0.230929,0.224952,1.056954,0.921730,-0.810815,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.751486,1.218393,0.231046,-0.756723,0.516833,0.275776,1.043127,-2.138114,0.232814,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


#### Preprocessed Validation data (Feature) 만들기

In [8]:
# 아웃풋 경로 지정
transform_validation_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-validation-output')
# scikit_learn_inferencee_model 에서 Validation Transformer 생성
transformer_validation = scikit_learn_inferencee_model.transformer(
    instance_count=1, 
    instance_type= instance_type,
    assemble_with = 'Line',
    output_path = transform_validation_output_path,
    accept = 'text/csv')
# Preprocess validation input
transformer_validation.transform(s3_input_validation.config['DataSource']['S3DataSource']['S3Uri'], content_type='text/csv')
print('Waiting for transform job: ' + transformer_validation.latest_transform_job.job_name)
transformer_validation.wait()
preprocessed_validation_path = transformer_validation.output_path+transformer_validation.latest_transform_job.job_name
print(preprocessed_validation_path)


Attaching to tmpx7p1km_d_algo-1-1c0gy_1
algo-1-1c0gy_1  | Processing /opt/ml/code
algo-1-1c0gy_1  | Building wheels for collected packages: preprocessing
algo-1-1c0gy_1  |   Building wheel for preprocessing (setup.py) ... done
algo-1-1c0gy_1  |   Created wheel for preprocessing: filename=preprocessing-1.0.0-py2.py3-none-any.whl size=9701 sha256=de867bcc809c8727d2acf7839f2f39832edc8725290a446359273fd5aa3e2f53
algo-1-1c0gy_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-8e40e_q2/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-1c0gy_1  | Successfully built preprocessing
algo-1-1c0gy_1  | Installing collected packages: preprocessing
algo-1-1c0gy_1  | Successfully installed preprocessing-1.0.0
algo-1-1c0gy_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-1c0gy_1  |   

In [9]:
preprocessed_validation_path

's3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-validation-output/sagemaker-scikit-learn-2020-07-19-01-59-2020-07-19-01-59-43-297'

In [10]:
! aws s3 ls {preprocessed_validation_path} --recursive

2020-07-19 01:59:49     300975 sagemaker/customer-churn/transformtrain-validation-output/sagemaker-scikit-learn-2020-07-19-01-59-2020-07-19-01-59-43-297/validation.csv.out


---
## Train with XGBoost
Built-in XGboost 알고리즘 컨테이너를 가져옵니다.

In [11]:
import boto3

from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost')

	get_image_uri(region, 'xgboost', '0.90-2').


S3에 있는 Train, Validation 전처리된 (Features) 데이타의 경로 및 파일 포맷등을 지정하는 오브젝트를 생성 합니다.

In [12]:
s3_input_train_processed = sagemaker.session.s3_input(
    preprocessed_train_path, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print(s3_input_train_processed.config)
s3_input_validation_processed = sagemaker.session.s3_input(
    preprocessed_validation_path, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print(s3_input_validation_processed.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-07-19-01-59-2020-07-19-01-59-35-904', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-validation-output/sagemaker-scikit-learn-2020-07-19-01-59-2020-07-19-01-59-43-297', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}


아래는 중요한 XGBoost의 하이퍼파라미터 입니다. 아래 내용 참고 하세요.
- `max_depth` controls how deep each tree within the algorithm can be built.  Deeper trees can lead to better fit, but are more computationally expensive and can lead to overfitting.  There is typically some trade-off in model performance that needs to be explored between a large number of shallow trees and a smaller number of deeper trees.
- `subsample` controls sampling of the training data.  This technique can help reduce overfitting, but setting it too low can also starve the model of data.
- `num_round` controls the number of boosting rounds.  This is essentially the subsequent models that are trained using the residuals of previous iterations.  Again, more rounds should produce a better fit on the training data, but can be computationally expensive or lead to overfitting.
- `eta` controls how aggressive each round of boosting is.  Larger values lead to more conservative boosting.
- `gamma` controls how aggressively trees are grown.  Larger values lead to more conservative models.

More detail on XGBoost's hyperparmeters can be found on their GitHub [page](https://github.com/dmlc/xgboost/blob/master/doc/parameter.md).

In [13]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container, # Built-in XGBoost Container
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train_processed, 'validation': s3_input_validation_processed}) 

2020-07-19 01:59:50 Starting - Starting the training job...
2020-07-19 01:59:52 Starting - Launching requested ML instances.........
2020-07-19 02:01:24 Starting - Preparing the instances for training...
2020-07-19 02:02:20 Downloading - Downloading input data...
2020-07-19 02:02:33 Training - Downloading the training image..Arguments: train
[2020-07-19:02:02:54:INFO] Running standalone xgboost training.
[2020-07-19:02:02:54:INFO] File size need to be processed in the node: 1.29mb. Available memory size in the node: 8488.67mb
[2020-07-19:02:02:54:INFO] Determined delimiter of CSV input is ','
[02:02:54] S3DistributionType set as FullyReplicated
[02:02:54] 2333x69 matrix with 160977 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2020-07-19:02:02:54:INFO] Determined delimiter of CSV input is ','
[02:02:54] S3DistributionType set as FullyReplicated
[02:02:54] 666x69 matrix with 45954 entries loaded from /opt/ml/input/data/validation?format=csv&label_co

## Post-processing
- XGBoost에서 나온 결과 값( 0 <= Score <=1) 을 0.5 이하이면 False, 이상이면 True로 변환 함

In [14]:
# sklearn_preprocessor 로 부터 후처리 모델을 생성 함.(scikit_learn_post_process_model)
# 다만 환경 변수를 바꾸어 후처리용으로 사용 함 ()'TRANSFORM_MODE': 'inverse-label-transform')
transform_postprocessor_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-postprocessing-output')
scikit_learn_post_process_model = sklearn_preprocessor.create_model(env={'TRANSFORM_MODE': 'inverse-label-transform'})
transformer_post_processing = scikit_learn_post_process_model.transformer(
    instance_count=1, 
    instance_type='local',
    assemble_with = 'Line',
    output_path = transform_postprocessor_path,
    accept = 'text/csv')

## Inference Pipeline <a class="anchor" id="pipeline_setup"></a>

![Inference-pipeline](img/Fig2.2.inference_pipeline.png)


Machine Learning pipeline 는 create_model() 를 호출하여 만들 수 있습니다. 예를 들어 여기서는 the fitted Scikit-learn inference model, the fitted Xgboost model and the psotprocessing model 의 세가지 모델을 가지고 만듦니다.

아래는 세개 모델의 위치를 확인 함.

In [15]:
print("Feature Transformer Model:\n {}".format(sklearn_preprocessor.model_data))
print("XGBoost Model:\n {}".format(xgb.model_data))
print("Post-Processing Model :\n {}".format(scikit_learn_post_process_model.model_data))

Feature Transformer Model:
 s3://sagemaker-us-east-2-057716757052/sagemaker-scikit-learn-2020-07-19-01-59-31-329/model.tar.gz
XGBoost Model:
 s3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/output/xgboost-2020-07-19-01-59-50-375/output/model.tar.gz
Post-Processing Model :
 s3://sagemaker-us-east-2-057716757052/sagemaker-scikit-learn-2020-07-19-01-59-31-329/model.tar.gz


In [17]:
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_name = 'churn-inference-pipeline-' + timestamp_prefix
client = boto3.client('sagemaker')
response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            'Image': sklearn_preprocessor.image_name,
            'ModelDataUrl': sklearn_preprocessor.model_data,
            'Environment': {
                    "SAGEMAKER_ENABLE_CLOUDWATCH_METRICS": str(sklearn_preprocessor.enable_cloudwatch_metrics),
                    "SAGEMAKER_SUBMIT_DIRECTORY": sklearn_preprocessor.uploaded_code.s3_prefix,
                    "TRANSFORM_MODE": "feature-transform",
                    "SAGEMAKER_CONTAINER_LOG_LEVEL": str(sklearn_preprocessor.container_log_level),
                    "SAGEMAKER_REGION": sklearn_preprocessor.sagemaker_session.boto_region_name,
                    "SAGEMAKER_PROGRAM": sklearn_preprocessor.uploaded_code.script_name
                }
        },
        {
            'Image': xgb.image_name,
            'ModelDataUrl': xgb.model_data,
            "Environment": {}
        },
        {
            'Image': scikit_learn_post_process_model.image,
            'ModelDataUrl': scikit_learn_post_process_model.model_data,
            'Environment': {
                    "SAGEMAKER_ENABLE_CLOUDWATCH_METRICS": str(sklearn_preprocessor.enable_cloudwatch_metrics),
                    "SAGEMAKER_SUBMIT_DIRECTORY": sklearn_preprocessor.uploaded_code.s3_prefix,
                    "TRANSFORM_MODE": "inverse-label-transform",
                    "SAGEMAKER_CONTAINER_LOG_LEVEL": str(sklearn_preprocessor.container_log_level),
                    "SAGEMAKER_REGION": sklearn_preprocessor.sagemaker_session.boto_region_name,
                    "SAGEMAKER_PROGRAM": sklearn_preprocessor.uploaded_code.script_name
                }
        },
    ],
    ExecutionRoleArn = role,
)
model_name

'churn-inference-pipeline-2020-07-19-02-04-24'

최종 Inference Pipeline의 모델을 확인 합니다.

In [18]:
print("Inference pipeline model name:\n {}".format(model_name))

Inference pipeline model name:
 churn-inference-pipeline-2020-07-19-02-04-24


In [19]:
%store model_name

Stored 'model_name' (str)
